In [ ]:
# import required libraries
from azure.ai.ml import MLClient

from azure.identity import DefaultAzureCredential

### Set Connection to Azure Machine Learning Workspace

In [ ]:
# enter details of your Azure Machine Learning workspace
subscription_id = 'xxx'
resource_group = 'xxx'
workspace = 'xxx'

# get a handle to the workspace
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

### Get the Existing Model

In [ ]:
# Create a Model object using the name and version of the already registered model
model = ml_client.models.get(name="bge-m3", version="1")

### Get the Existing Environment

In [ ]:
env = ml_client.environments.get(name="huggingface-inference-gpu", version=2)

### Get the Existing Endpoint

In [ ]:
endpoint_name = ml_client.online_endpoints.get(name="hugging-face-embeddings-endpoint").name

### Set new Parameters for the Deployment

Here, we can change any parameters of the deployment or even pass an updated scoring script.

In [ ]:
from azure.ai.ml.entities import (OnlineRequestSettings,
                                  ManagedOnlineDeployment,
                                  CodeConfiguration)
deployment = ManagedOnlineDeployment(
    name="hugging-face-embeddings-deployment-1", 
    endpoint_name=endpoint_name,
    model=model,
    environment=env,
    code_configuration=CodeConfiguration(
        code="path/to/code/folder", scoring_script="score.py"
    ),
    instance_type="Standard_NC4as_T4_v3",
    instance_count=1,
    request_settings=OnlineRequestSettings(
        max_concurrent_requests_per_instance = 100,
        request_timeout_ms=180000, # max setting is 180000 (3 minutes)
        max_queue_wait_ms=300000),
)

### Update the Deployment

In [ ]:
ml_client.online_deployments.begin_create_or_update(deployment)